In [12]:
# Group 4 - Apriori Algorithm for Cross selling association rules 
# Introduction:
# This script performs data preprocessing and association rule mining on a sales dataset.
# The main objective is to identify interesting associations between different 'PRH4' categories within customer orders.
# The script involves the following steps:
# 1. Importing necessary libraries.
# 2. Loading the dataset and cleaning it by removing customers with null values.
# 3. Filtering customers based on specific criteria related to unique orders and PRH4 categories.
# 4. Exporting the cleaned dataset for further analysis.
# 5. Loading the cleaned dataset and transforming it for association rule mining.
# 6. Applying the Apriori algorithm to find frequent itemsets.
# 7. Generating association rules from the frequent itemsets.
# 8. Calculating Zhang's metric to evaluate the strength of the rules.
# 9. Exporting the resulting association rules to a CSV file.

# Summary:
# The script successfully cleans the dataset by removing customers with null values and those who don't meet specific criteria.
# The cleaned dataset is exported for further analysis and then transformed for association rule mining.
# The Apriori algorithm identifies frequent itemsets with a minimum support threshold of 0.005.
# Association rules are generated with a minimum confidence threshold of 0.1, revealing interesting patterns in the data.
# Zhang's metric is calculated for each rule to evaluate its strength and relevance.
# The resulting association rules are exported to a CSV file for further interpretation and use.
# The script provides insights into customer ordering behaviors and the relationships between different 'PRH4' categories.


In [1]:
import pandas as pd
import glob
import os
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from scipy.stats import randint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from mlxtend.frequent_patterns import apriori, association_rules

In [8]:
# Load the dataset
file_path = './Data/MergedData.csv'
merged_df = pd.read_csv(file_path)

# Identify customers with any NULL values in any column
customers_with_nulls = merged_df[merged_df.isnull().any(axis=1)]['customer_number'].unique()

# Remove all records of these customers from the dataset
cleaned_df = merged_df[~merged_df['customer_number'].isin(customers_with_nulls)]

# Calculate the unique order count for each customer
unique_orders_per_customer = cleaned_df.groupby('customer_number')['order_number'].nunique()

# Identify customers who have placed more than 3 unique orders
customers_with_more_than_3_orders = unique_orders_per_customer[unique_orders_per_customer > 2].index

# Filter the cleaned_df to keep only those customers
filtered_cleaned_df = cleaned_df[cleaned_df['customer_number'].isin(customers_with_more_than_3_orders)]

# Calculate the number of unique PRH4 per order
unique_prh4_per_order = filtered_cleaned_df.groupby(['order_number', 'customer_number'])['PRH4'].nunique()

# Identify customers who have ordered more than 7 unique PRH4 in the same order
customers_with_more_than_7_prh4 = unique_prh4_per_order[unique_prh4_per_order >= 7].index.get_level_values('customer_number').unique()

# Remove all records of these customers from the filtered_cleaned_df
final_cleaned_df = filtered_cleaned_df[~filtered_cleaned_df['customer_number'].isin(customers_with_more_than_7_prh4)]

# Display the number of records in the final cleaned dataset
num_records_final_cleaned = final_cleaned_df.shape[0]
print(f'The number of records in the final_cleaned_df DataFrame: {num_records_final_cleaned}')

# Check the number of unique customers in the final cleaned dataframe
unique_customers_final_cleaned = final_cleaned_df['customer_number'].nunique()
print(f'The number of unique customers in the final_cleaned_df DataFrame: {unique_customers_final_cleaned}')

# Export the final_cleaned_df to the specified location
output_file_path = './Data/final_cleaned_data.csv'
final_cleaned_df.to_csv(output_file_path, index=False)

print(f'Final cleaned dataframe has been exported to: {output_file_path}')

/var/folders/hl/w5x9js51325d0gtv9ldxkd8c0000gn/T/ipykernel_93770/2537017791.py:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv(file_path)


The number of records in the final_cleaned_df DataFrame: 94995
The number of unique customers in the final_cleaned_df DataFrame: 5003
Final cleaned dataframe has been exported to: /Users/sachin/Documents/MPS/Courses/ALY6980/Client Data/TempCSVs/final_cleaned_data.csv


In [10]:


# Load the cleaned dataframe
preprocessed_file_path = './Data/final_cleaned_data.csv'
df_step3 = pd.read_csv(preprocessed_file_path)

# Transform data: Group transactions by order_number and list all PRH4 products in each order
transactions = df_step3.groupby('order_number')['PRH4'].apply(lambda x: list(set(x))).tolist()

# Create a DataFrame for the transactions with one-hot encoding
all_items = sorted(set(item for sublist in transactions for item in sublist))
one_hot_encoded_df = pd.DataFrame([[item in transaction for item in all_items] for transaction in transactions], columns=all_items)

# Apply the Apriori algorithm
frequent_itemsets = apriori(one_hot_encoded_df, min_support=0.005, use_colnames=True)

# Print frequent itemsets to debug
print("Frequent Itemsets:")
print(frequent_itemsets)

# Generate association rules with a lower confidence threshold
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)

# Print rules to debug
print("Association Rules:")
print(rules)

# Calculate Zhang's metric
rules['zhangs_metric'] = (rules['confidence'] - rules['support']) / (1 - rules['support'])

# Select and rename relevant columns
rules = rules[['antecedents', 'consequents', 'antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'leverage', 'conviction', 'zhangs_metric']]
rules.columns = ['antecedents', 'consequents', 'antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'leverage', 'conviction', 'zhangs_metric']

# Convert frozenset to string for easier reading
rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

# Export the results to a CSV file
output_file_path = './Data/PRH4_association_rules.csv'
rules.to_csv(output_file_path, index=False)

print(f'Association rules have been exported to: {output_file_path}')


Frequent Itemsets:
     support                                           itemsets
0   0.040824                            (Abutments, Edentulous)
1   0.048773                                   (Abutments, SRA)
2   0.045548                               (Abutments, Ti Base)
3   0.027628                 (Abutments, single tooth + bridge)
4   0.033917                       (Allogenic Bone Substitutes)
5   0.126967                              (BL Healing Surgical)
6   0.070537                    (BLAT Ti Implants, hydrophilic)
7   0.072329                       (BLAT Ti Implants, standard)
8   0.046362                  (BLAT TiZr Implants, hydrophilic)
9   0.027042                     (BLAT TiZr Implants, standard)
10  0.031929               (BLFT TiZr Implants, hydrophilic TF)
11  0.011859                  (BLFT TiZr Implants, standard TF)
12  0.008341                  (BLPW TiZr Implants, hydrophilic)
13  0.015248                                          (Copings)
14  0.007559         